In [ ]:
import pandas as pd
import numpy as np

<h2> Getting data from Github Repository by Johns Hopkins CSSE

In [ ]:
meta = ["recovered", "confirmed","deaths"]
data = [pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_{}_global.csv".format(i),error_bad_lines=False)  for i in meta]

<H2> Defining countries in scope

In [ ]:
scope = ["Turkey","Italy","China"]

In [ ]:
limit1 = len(data[2].columns)
limit2 = len(data[1].columns)

<H2> Merging different regions of individual countries

In [ ]:
scopeDataD = []

for i in scope:
    data_ = data[2][data[2]["Country/Region"] == i].iloc[: , list(range(4,limit1))].transpose().reset_index()
    sum_row = data_.sum(axis=1)
    data_["sum"] = pd.DataFrame(sum_row)
    data_ = data_[["index","sum"]]
    data_.columns = ["Date","Quantity"]
    scopeDataD.append(data_)

scopeDataC = []

for i in scope:
    data_ = data[1][data[1]["Country/Region"] == i].iloc[: , list(range(4,limit2))].transpose().reset_index()
    sum_row = data_.sum(axis=1)
    data_["sum"] = pd.DataFrame(sum_row)
    data_ = data_[["index","sum"]]
    data_.columns = ["Date","Quantity"]
    scopeDataC.append(data_)

In [ ]:
for i in range(len(scopeDataC)):
    scopeDataC[i]["Deaths"] = scopeDataD[i]["Quantity"]
    scopeDataC[i]["Rate"] = (pd.to_numeric(scopeDataC[i]["Deaths"])/pd.to_numeric(scopeDataC[i]["Quantity"]))*100
    scopeDataC[i]
for i in range(len(scopeDataC)):
    scopeDataC[i] = scopeDataC[i].fillna(0)

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (18,10)

In [ ]:
DaillyCase = []
QuaC = scopeDataC[1].Quantity
for i in range(len(QuaC)):
    if (QuaC[i] == 0) or i ==0:
        DaillyCase.append(QuaC[i])
    else:
        DaillyCase.append(QuaC[i]-QuaC[i-1])
        
        

In [ ]:
DaillyDeaths = []
QuaD = scopeDataD[1].Quantity
for i in range(len(QuaD)):
    if (QuaD[i] == 0) or i ==0:
        DaillyDeaths.append(QuaD[i])
    else:
        DaillyDeaths.append(QuaD[i]-QuaD[i-1])

<h2>Visualization

In [ ]:
t=0
for i in range(len(DaillyCase)):
    fig = plt.figure()
    
    ax1 = plt.subplot2grid((3, 3), (0, 0), colspan=3)

    plt.plot(scopeDataC[2].index[:i], scopeDataC[1].Rate[:i],linewidth=4,color="k")
    plt.annotate("Death Rate %{}".format(np.around(scopeDataC[1].Rate[i]),decimals=1),xy=(10,10),size=18)
    plt.title("Italy {}".format(scopeDataC[1].Date[i]), size=20)
    plt.ylabel("(Total Deaths / Total Cases)*100",size=12)
    plt.xlim(0,80)
    plt.ylim(0,20)

    ax2 = plt.subplot2grid((3, 3), (1, 0), colspan=3)

    plt.bar(scopeDataC[1].index[:i], DaillyCase[:i], color="y")
    plt.annotate("Total Cases: {}".format(scopeDataC[1].Quantity[i]),xy=(10,4000),size=18)
    if i >= 60:
        plt.annotate("Dailly Case Peak: {}".format(scopeDataC[1].Date[59]),xy=(59,6000),
                                 xycoords='data',
            xytext=(40, 4000), textcoords='data',
            size=15, va="center", ha="center",
            arrowprops=dict(arrowstyle="simple",
                           connectionstyle="arc3,rad=-0.2"))
    plt.ylabel("# ", size=12)
    plt.xlim(0,80)
    plt.ylim(0,7400)

    ax4 = plt.subplot2grid((3, 3), (2, 0), colspan=3)

    plt.bar(scopeDataC[1].index[:i],  DaillyDeaths[:i], color="r")
    plt.annotate("Total Deaths: {}".format(scopeDataD[1].Quantity[i]),xy=(10,600),size=18)
    if i >= 66:
        plt.annotate("Dailly Death Peak: {}".format(scopeDataC[1].Date[66]),xy=(65,800),
                                 xycoords='data',
            xytext=(40, 600), textcoords='data',
            size=15, va="center", ha="center",
            arrowprops=dict(arrowstyle="simple",
                            connectionstyle="arc3,rad=-0.2"))
    plt.ylabel("# ",size=12)
    plt.xlim(0,80)
    plt.ylim(0,1200)
    plt.xticks(rotation='vertical')

    plt.tight_layout()
    for i in range(3):
        plt.savefig("image/foto{}.png".format(t))
        t = t + 1
    
    

In [ ]:
import os
import subprocess
os.getcwd()
os.chdir('/Desktop/Dev/JupyterNotebooks/Covid-19/image')
subprocess.call(['ffmpeg', '-r', '10', '-f', 'image2', '-s', '800x600', '-i', 
                 'foto%01d.png', '-vcodec', 'libx264', '-crf', '24', '-pix_fmt', 'yuv420p', 'regression.mp4'])

ffmpeg -framerate 4 -i foto%01d.png -c:v libx264 -profile:v high -crf 20 -pix_fmt yuv420p output.mp4